In [1]:
import pandas as pd
import json
import numpy as np

In [2]:
train = pd.read_json('medmcqa_train.json', lines=True)

In [3]:
train

,question,exp,cop,opa,opb,opc,opd,subject_name,topic_name,id,choice_type
0,Chronic urethral obstruction due to benign pri...,Chronic urethral obstruction because of urinar...,3,Hyperplasia,Hyperophy,Atrophy,Dyplasia,Anatomy,Urinary tract,e9ad821a-c438-4965-9f77-760819dfa155,single
1,Which vitamin is supplied from only animal sou...,Ans. (c) Vitamin B12 Ref: Harrison's 19th ed. ...,3,Vitamin C,Vitamin B7,Vitamin B12,Vitamin D,Biochemistry,Vitamins and Minerals,e3d3c4e1-4fb2-45e7-9f88-247cc8f373b3,single
2,All of the following are surgical options for ...,"Ans. is 'd' i.e., Roux en Y Duodenal Bypass Ba...",4,Adjustable gastric banding,Biliopancreatic diversion,Duodenal Switch,Roux en Y Duodenal By pass,Surgery,Surgical Treatment Obesity,5c38bea6-787a-44a9-b2df-88f4218ab914,multi
3,Following endaerectomy on the right common car...,The central aery of the retina is a branch of ...,1,Central aery of the retina,Infraorbital aery,Lacrimal aery,Nasociliary aretry,Ophthalmology,None,cdeedb04-fbe9-432c-937c-d53ac24475de,multi
4,Growth hormone has its effect on growth through?,"Ans. is 'b' i.e., IGI-1GH has two major functi...",2,Directly,IG1-1,Thyroxine,Intranuclear receptors,Physiology,None,dc6794a3-b108-47c5-8b1b-3b4931577249,single
...,...,...,...,...,...,...,...,...,...,...,...
182817,Most common site for extra mammary Paget&;s di...,.It is superficial manifestation of an intradu...,1,Vulva,Vagina,Penis,Anus,Surgery,Urology,04519631-61e3-4ef4-b2d6-c352cf349ded,single
182818,Inferior Rib notching is seen in all except?,Answer is D (Neurofibromatosis) Neurofibromato...,4,Coarctation of aoa,Classical blalock tausing operation,SVC obstruction,Neurofibromatosis,Medicine,None,fa6b40a1-459a-401c-96e7-fac5608e7744,multi
182819,Which is false regarding cryptococcus neoformans?,"Ans. is 'c' i e., Urease negative Cryptococcus...",3,Grows at 5degC and 37degC,It has 4 serotypes,Urease negative,Causes superficial skin infection,Microbiology,None,4ecebffe-72f6-4988-8d64-738e43ad6086,multi
182820,Histopathological finding of gluten hypersensi...,"Ans. is 'a' i.e., Crypt hyperplasia Histopatho...",1,Crypt hyperplasia,Increase in thickness of the mucosa,Distal intestine involvement,Villous hypertrophy,Pathology,G.I.T.,7cf99676-f2b5-4f6d-9cf5-03ff7651afb3,single


In [4]:
def filter_qa(x):
    try:
        c_idx = int(x['cop'])-1
        if c_idx > 3 or c_idx < 0:
            return np.nan
        answer = [x['opa'],x['opb'],x['opc'],x['opd']][c_idx]
        txt = 'Question: ' + x['question'] + \
            "\nAnswer: " + answer + "\nExplanation: " + x['exp'] 
        return txt
    except Exception:
        return np.nan

just_text = train.apply(lambda x: filter_qa(x), axis=1)

In [5]:
print(filter_qa(train.iloc[0]))

Question: Chronic urethral obstruction due to benign prismatic hyperplasia can lead to the following change in kidney parenchyma
Answer: Atrophy
Explanation: Chronic urethral obstruction because of urinary calculi, prostatic hyperophy, tumors, normal pregnancy, tumors, uterine prolapse or functional disorders cause hydronephrosis which by definition is used to describe dilatation of renal pelvis and calculus associated with progressive atrophy of the kidney due to obstruction to the outflow of urine Refer Robbins 7yh/9,1012,9/e. P950


In [6]:
just_text_clean = just_text.dropna(inplace=False).reset_index()
just_text_clean = just_text_clean.drop(['index'], axis=1)
just_text_clean = just_text_clean.rename(columns={0:'text'})

In [7]:
just_text_clean.to_csv('medmcqa_for_pretraining.csv')

In [8]:
def filter_qa(x):
    try:
        c_idx = int(x['cop'])-1
        if c_idx > 3 or c_idx < 0:
            return np.nan
        answer = [x['opa'],x['opb'],x['opc'],x['opd']][c_idx]
        txt = 'Question: ' + x['question'] + \
            "\nAnswer: " + answer + "\nExplanation: " + x['exp'] 
        return txt
    except Exception:
        return np.nan

questions = []
answer_words = []
answer_letters = []
explanations = []
topics = []
subjects = []
for idx, row in train.iterrows():
    try:
        c_idx = int(row['cop'])-1
        if c_idx > 3 or c_idx < 0:
            questions.append(np.nan)
            answer_words.append(np.nan)
            answer_letters.append(np.nan)
            explanations.append(row['exp'])
            topics.append(row['topic_name'])
            subjects.append(row['subject_name'])
            continue

        question = row['question'] + \
            "\n(A) " + row['opa'] + \
            "\n(B) " +row['opb'] + \
            "\n(C) " +row['opc'] + \
            "\n(D) " +row['opd']
        answer_letter = ['A','B','C','D'][c_idx]
        answer_word = [row['opa'],row['opb'],row['opc'],row['opd']][c_idx]
        
        questions.append(question)
        answer_words.append(answer_word)
        answer_letters.append(answer_letter)
        explanations.append(row['exp'])
        topics.append(row['topic_name'])
        subjects.append(row['subject_name'])
    except Exception:
        answer_words.append(np.nan)
        answer_letters.append(np.nan)
        explanations.append(row['exp'])
        topics.append(row['topic_name'])
        subjects.append(row['subject_name'])

In [9]:
df = pd.DataFrame(
    {
        'question': questions,
        'answer_word': answer_words,
        'answer_letter': answer_letters,
        'explanation': explanations,
        'topic': topics,
        'subject': subjects,
    }
)

In [10]:
len(df)

182822

In [11]:
df = df.dropna(axis=0)

In [12]:
len(df)

87077

In [13]:
# df = df.drop('Unnamed: 0', axis=1)
df.to_csv('medmcqa_train.csv', index=False)

In [14]:
df = pd.read_csv('medmcqa_train.csv')
df

,question,answer_word,answer_letter,explanation,topic,subject
0,Chronic urethral obstruction due to benign pri...,Atrophy,C,Chronic urethral obstruction because of urinar...,Urinary tract,Anatomy
1,Which vitamin is supplied from only animal sou...,Vitamin B12,C,Ans. (c) Vitamin B12 Ref: Harrison's 19th ed. ...,Vitamins and Minerals,Biochemistry
2,All of the following are surgical options for ...,Roux en Y Duodenal By pass,D,"Ans. is 'd' i.e., Roux en Y Duodenal Bypass Ba...",Surgical Treatment Obesity,Surgery
3,Per rectum examination is not a useful test fo...,Pilonidal sinus,C,PILONIDAL SINUS/DISEASE (Jeep Bottom; Driver's...,Urology,Surgery
4,Naglers reaction is shown by\n(A) Clostridium ...,Clostridium perfringens,C,Nagler's reaction - when Clostridium perfringe...,Bacteriology,Microbiology
...,...,...,...,...,...,...
87072,Which of the following indicates anticipated d...,Beard,C,Predicting difficulty in mask ventilation Mask...,Anaesthesia Q Bank,Anaesthesia
87073,Enzyme replacement therapy is used for-\n(A) G...,Gaucher's disease,A,"Ans. is 'a' i.e., Gaucher's disease * Alglucer...",Enzymes,Biochemistry
87074,Organism that causes emphysematous cholecystit...,Clostridium perfringens,C,Ref: Harrison's 18th editionExplanation:Emphys...,Gall Bladder,Medicine
87075,Most common site for extra mammary Paget&;s di...,Vulva,A,.It is superficial manifestation of an intradu...,Urology,Surgery
